In [1]:
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
from kaveh.behavioral import oculomotor
from kaveh.sorting import spikesorter
from kaveh.toolbox import find_file
import numpy as np
from matplotlib import pyplot as plt
from smr import File
import csv
import os

In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [3]:
raw_files_with_8_dir_exp = []
num_clust = []
with open('/mnt/data/temp/kaveh/2010_Adapt.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if(row[1] == '-'):
            path, filename = os.path.split(row[0])
            f_name = find_file(filename[:-4], '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/')
            raw_files_with_8_dir_exp = raw_files_with_8_dir_exp + [f_name]
            num_clust = num_clust + [int(row[2])]
            

In [4]:
# print(raw_files_with_8_dir_exp[0])

In [5]:
cs_path = '/mnt/data/temp/kaveh/auto_processed/2010_Adapt' 
cs_files = [find_file(os.path.split(fn)[1] + '.pkl', cs_path) for fn in raw_files_with_8_dir_exp]


In [6]:
# # Read detected CS
# f_index = 0
# with open(cs_files[f_index], 'rb') as input:
#     sss = pickle.load(input)

In [7]:
# smr_content = File(raw_files_with_8_dir_exp[f_index])
# smr_content.read_channels()
# chan_titles = [chan.title for chan in smr_content.channels]
# for i, ct in enumerate(chan_titles):
#     print("{}: {}".format(i, ct))

In [8]:
# vt = smr_content.get_channel(4)
# # ve = smr_content.get_channel(2)
# ht = smr_content.get_channel(3)
# # he = smr_content.get_channel(1)

# voltage = smr_content.get_channel(0).data

# # t_range = (0,700) #s
# # fsize =  (20,5)
# # i_range = [int(x/vt.dt) for x in t_range]
# # prange = slice(i_range[0], i_range[1])

# tt = oculomotor.target(vt.data, ht.data, ht.dt, '2d')
# target_jumps_to, jump_vecs = tt.get_target_jumps(num_clusters=16, jump_tol= 150)


In [9]:
# print(voltage.size*sss.dt)
# print(tt.ht.size*tt.dt)

In [10]:
def convert_index(source_dt, dest_dt, source_index):
    """
    converts an index in a signal with source_dt period to an index in a signal with dest_dt period
    """
    t = source_index*source_dt
    dest_index = np.round(t/dest_dt)
    return dest_index

In [11]:
# target_jumps_to_converted = {}
# for jv in jump_vecs:
#     target_jumps_to_converted[str(jv)] = convert_index(tt.dt, sss.dt, target_jumps_to[str(jv)])

In [12]:
# cs_indices = sss.get_cs_spike_indices()


In [13]:
# search_range = (0.050, 0.250) #in seconds; sets the range of time to look for a cs after target jump
# i_range = [int(x/sss.dt) for x in search_range]
# for jv in jump_vecs:
#     target_indices = target_jumps_to_converted[str(jv)]
#     for ti in target_indices:
#         i_range = ti + i_range
#         if np.any(np.logical_and(cs_indices <= i_range[1] , cs_indices >= i_range[0])):
#             print(jv, i_range)
            
            
        

In [14]:
# from neo.io import Spike2IO
from kaveh.toolbox import closest_argmin

search_range = (0.050, 0.200) #in seconds; sets the range of time to look for a cs after target jump
for i, cf in enumerate(cs_files):
    print(cf)
    with open(cf, 'rb') as input:
        sss = pickle.load(input)
    print(raw_files_with_8_dir_exp[i])
    i_range = [int(x/sss.dt) for x in search_range]

    smr_content = File(raw_files_with_8_dir_exp[i])
    smr_content.read_channels()

#     neo_reader = Spike2IO(filename=raw_files_with_8_dir_exp[i])
#     neo_data = neo_reader.read()
#     data_block = neo_data[0]
#     seg = data_block.segments[0]
#     # Assuming cs spike trains are saved in the 0th index of spike train channels
#     cs_spiketrain = np.array(seg.spiketrains[0])
#     cs_spiketrain = np.int32(cs_spiketrain/sss.dt)
    
#     print('number of complex spikes from file ',cs_spiketrain.shape)

    vt = smr_content.get_channel(4)
    ht = smr_content.get_channel(3)
    voltage = smr_content.get_channel(0).data

    tt = oculomotor.target(vt.data, ht.data, ht.dt, '2d')
    target_jumps_to, jump_vecs = tt.get_target_jumps(num_clusters=num_clust[i], jump_tol= 150)
    
    target_jumps_to_converted = {}
    for jv in jump_vecs:
        target_jumps_to_converted[str(jv)] = convert_index(tt.dt, sss.dt, target_jumps_to[str(jv)])
    cs_indices = sss.get_cs_spike_indices()
    print('number of complex spikes from alg ', cs_indices.shape)
#     print('number of simple spikes from alg ', sss.get_spike_indices())
    
    for jv in jump_vecs:
        target_indices = target_jumps_to_converted[str(jv)]
        print('number of target jumps to ' + str(jv) + ' = ' + str(target_indices.size) )
        if target_indices.size > 5:
            cs_counter = 0;
            for ti in target_indices:
                i_range = ti + i_range
                if np.any(np.logical_and(cs_indices <= i_range[1] , cs_indices >= i_range[0])):
                    cs_counter = cs_counter + 1
                    print(jv, i_range)
            probability = float(cs_counter) / target_indices.size
            if probability != 0:
                print('------------------> probability of cs, ', probability)

/mnt/data/temp/kaveh/auto_processed/2010_Adapt/Wyatt/W120508_2/W120508_2_1548_Adapt.smr.pkl
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Wyatt/W120508_2/W120508_2_1548_Adapt.smr
Not implemented (type = 6)
Not implemented (type = 6)
Not implemented (type = 6)
Not implemented (type = 5)
Not implemented (type = 5)
('number of complex spikes from alg ', (869,))
number of target jumps to [-1966     0] = 486
number of target jumps to [3932    0] = 927
number of target jumps to [-3932     0] = 842
number of target jumps to [1579    0] = 478
number of target jumps to [-8193     0] = 30
number of target jumps to [8193    0] = 28
number of target jumps to [2912    0] = 1
number of target jumps to [-3277     0] = 54
number of target jumps to [-2590     0] = 48
number of target jumps to [-1638     0] = 148
/mnt/data/temp/kaveh/auto_processed/2010_Adapt/Wyatt/W120208_2/W120208_1531_Adapt.smr.pkl
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Wyatt/W120208_2/W120208_1531_Ad

Not implemented (type = 6)
Not implemented (type = 6)
Not implemented (type = 5)
Not implemented (type = 5)
('number of complex spikes from alg ', (353,))
number of target jumps to [1639    0] = 344
number of target jumps to [-3932     0] = 425
number of target jumps to [3934    0] = 427
number of target jumps to [-1639     0] = 344
number of target jumps to [-3277     0] = 50
number of target jumps to [2621    0] = 46
number of target jumps to [-2621     0] = 47
number of target jumps to [3277    0] = 45
number of target jumps to [  -1 3276] = 1
number of target jumps to [1966    0] = 103
number of target jumps to [-1966     0] = 102
/mnt/data/temp/kaveh/auto_processed/2010_Adapt/Wyatt/W120908_1/W120908_1_1524_Adapt.smr.pkl
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Wyatt/W120908_1/W120908_1_1524_Adapt.smr
Not implemented (type = 6)
Not implemented (type = 6)
Not implemented (type = 6)
Not implemented (type = 5)
Not implemented (type = 5)
('number of complex spikes fro

In [15]:
# plt.figure()
# plt.ylim(-8000, 8000)
# plt.xlim(-8000, 8000)

# plt.plot(jump_vecs[:,0], jump_vecs[:,1], '.')


In [16]:
# plt.figure(figsize=fsize)
# plt.plot(tt.vt, alpha=0.3)
# plt.ylim(-20000, 40000)

# a = plt.axes([0.45, 0.60, 0.2, 0.2], facecolor = 'r', aspect='equal')
# from kaveh.plots import axvlines
# colors = plt.cm.jet(np.linspace(0, 1, len(jump_amps)))
# # for i, jv in enumerate(jump_vecs):
# #     axvlines(plt.gca(), target_jumps_to_converted[jv], color=colors[i])
# # axvlines(plt.gca(), sss.get_cs_spike_indices(), color='m', alpha = 0.3)


In [17]:
# for jv in jump_vecs:
#     print(jv)